In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

In [3]:
import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

ts_info_fullfn = debris_prms.ts_fp + debris_prms.roi + '_debris_tsinfo.nc'

if os.path.exists(debris_prms.ts_fp) == False:
    os.makedirs(debris_prms.ts_fp)
    
debug = False

print(ts_info_fullfn)

/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/05_ts_data/05_debris_tsinfo.nc


In [4]:
rgiid_list = []
rgiid_fn_list = []
for i in os.listdir(debris_prms.mb_binned_fp):
    if i.endswith('mb_bins.csv'):
        region = int(i.split('.')[0])
        if region in debris_prms.roi_rgidict[debris_prms.roi]:    
            if region < 10:
                rgiid_list.append(i[0:7])
            else:
                rgiid_list.append(i[0:8])
            rgiid_fn_list.append(i)
        

# rgiid_list = ['18.00718']
# rgiid_fn_list = ['18.00718_mb_bins.csv']

rgiid_list = sorted(rgiid_list)
rgiid_fn_list = sorted(rgiid_fn_list)



main_glac_rgi = debris_prms.selectglaciersrgitable(rgiid_list)
main_glac_rgi['CenLon_360'] = main_glac_rgi['CenLon']
main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'])
main_glac_rgi['bin_fn'] = rgiid_fn_list

611 glaciers in region 5 are included in this model run: ['00115', '00181', '00184', '00258', '00270', '00275', '00318', '00319', '00323', '00334', '00335', '00336', '00337', '00342', '00400', '00421', '00445', '00446', '00451', '00459', '00460', '00522', '00530', '00789', '00790', '00793', '00794', '00800', '00801', '00817', '00839', '00859', '00879', '00904', '00938', '01224', '01355', '01383', '01403', '01441', '01452', '01456', '01706', '01780', '02398', '02521', '02536', '02918', '03019', '03185'] and more
This study is focusing on 611 glaciers in region [5]


In [5]:
# Merge with debris cover stats
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_%'].values))
main_glac_rgi['DC_Area_%'] = main_glac_rgi.RGIId.map(dc_areaperc_dict).fillna(0)
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))
main_glac_rgi['DC_Area_v2'] = main_glac_rgi.RGIId.map(dc_area_dict).fillna(0)
main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,114,RGI60-05.00115,-52.1262,65.6000,5,1,72.377,143,1139,825,...,0,9,20000823,115,05.00115,5.00115,307.8738,5.00115_mb_bins.csv,2.86,1100729
1,180,RGI60-05.00181,-52.9288,65.7104,5,1,2.060,111,1015,640,...,0,9,20000823,181,05.00181,5.00181,307.0712,5.00181_mb_bins.csv,6.60,108966
2,183,RGI60-05.00184,-52.9527,65.6998,5,1,2.807,241,925,625,...,0,9,20000814,184,05.00184,5.00184,307.0473,5.00184_mb_bins.csv,12.18,232227
3,257,RGI60-05.00258,-52.5941,65.8403,5,1,4.232,378,1526,1010,...,0,9,20000823,258,05.00258,5.00258,307.4059,5.00258_mb_bins.csv,8.46,253069
4,269,RGI60-05.00270,-52.5381,65.8321,5,1,5.630,351,1663,1062,...,0,9,20000823,270,05.00270,5.00270,307.4619,5.00270_mb_bins.csv,6.41,288195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,20158,RGI60-05.20173,-21.0378,74.7637,5,1,3.393,827,1292,1062,...,0,9,20010820,20173,05.20173,5.20173,338.9622,5.20173_mb_bins.csv,10.40,183634
607,20168,RGI60-05.20183,-22.6020,74.9837,5,1,3.712,1223,1402,1294,...,0,9,20010820,20183,05.20183,5.20183,337.3980,5.20183_mb_bins.csv,13.31,353063
608,20183,RGI60-05.20198,-22.0165,74.9015,5,1,45.809,923,1649,1297,...,0,9,20010820,20198,05.20198,5.20198,337.9835,5.20198_mb_bins.csv,5.59,2336088


In [6]:
# Latitude and longitude index to run the model
#  Longitude must be 0 - 360 degrees
latlon_all = []
for i in os.listdir(debris_prms.ostrem_fp):
    if i.endswith(debris_prms.ostrem_fn_sample.split('XXXX')[1]):
        latlon_fn = i.split(debris_prms.ostrem_fn_sample.split('XXXX')[1])[0]
        # Extract latitude
        lat_str = latlon_fn.split('-')[0]
        if 'N' in lat_str:
            i_lat = int(lat_str.split('N')[0]) / 100
        elif 'S' in lat_str:
            i_lat = -1 * int(lat_str.split('S')[0]) / 100
        # Extract longitude
        lon_str = latlon_fn.split('-')[1]
        i_lon = int(lon_str.split('E')[0]) / 100
        latlon_all.append([i_lat, i_lon, i])
latlon_all = sorted(latlon_all)

lat_all = np.array([x[0] for x in latlon_all])
lon_all = np.array([x[1] for x in latlon_all])
ostrem_fn_all_raw = [x[2] for x in latlon_all]

main_glac_rgi['lat_nearest'] = np.nan
main_glac_rgi['lon_nearest'] = np.nan
main_glac_rgi['ostrem_fn'] = np.nan
for nglac, glac_idx in enumerate(main_glac_rgi.index.values):
    latlon_dist = (((main_glac_rgi.loc[glac_idx,'CenLat'] - lat_all)**2 + 
                    (main_glac_rgi.loc[glac_idx,'CenLon_360'] - lon_all)**2)**0.5)
    latlon_nearidx = np.where(latlon_dist == latlon_dist.min())[0][0]
    
    main_glac_rgi.loc[glac_idx,'lat_nearest'] = lat_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'lon_nearest'] = lon_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'ostrem_fn'] = ostrem_fn_all_raw[latlon_nearidx]
    
ostrem_fn_all = sorted(list(np.unique(main_glac_rgi['ostrem_fn'].values)))
lat_values = np.arange(main_glac_rgi.lat_nearest.min(), main_glac_rgi.lat_nearest.max() + 0.1, 0.25)
lon_values = np.arange(main_glac_rgi.lon_nearest.min(), main_glac_rgi.lon_nearest.max() + 0.1, 0.25)

main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2,lat_nearest,lon_nearest,ostrem_fn
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,114,RGI60-05.00115,-52.1262,65.6000,5,1,72.377,143,1139,825,...,115,05.00115,5.00115,307.8738,5.00115_mb_bins.csv,2.86,1100729,65.50,307.75,6550N-30775E-debris_melt_curve.nc
1,180,RGI60-05.00181,-52.9288,65.7104,5,1,2.060,111,1015,640,...,181,05.00181,5.00181,307.0712,5.00181_mb_bins.csv,6.60,108966,65.75,307.00,6575N-30700E-debris_melt_curve.nc
2,183,RGI60-05.00184,-52.9527,65.6998,5,1,2.807,241,925,625,...,184,05.00184,5.00184,307.0473,5.00184_mb_bins.csv,12.18,232227,65.75,307.00,6575N-30700E-debris_melt_curve.nc
3,257,RGI60-05.00258,-52.5941,65.8403,5,1,4.232,378,1526,1010,...,258,05.00258,5.00258,307.4059,5.00258_mb_bins.csv,8.46,253069,65.75,307.50,6575N-30750E-debris_melt_curve.nc
4,269,RGI60-05.00270,-52.5381,65.8321,5,1,5.630,351,1663,1062,...,270,05.00270,5.00270,307.4619,5.00270_mb_bins.csv,6.41,288195,65.75,307.50,6575N-30750E-debris_melt_curve.nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,20158,RGI60-05.20173,-21.0378,74.7637,5,1,3.393,827,1292,1062,...,20173,05.20173,5.20173,338.9622,5.20173_mb_bins.csv,10.40,183634,75.00,339.75,7500N-33975E-debris_melt_curve.nc
607,20168,RGI60-05.20183,-22.6020,74.9837,5,1,3.712,1223,1402,1294,...,20183,05.20183,5.20183,337.3980,5.20183_mb_bins.csv,13.31,353063,75.00,338.00,7500N-33800E-debris_melt_curve.nc
608,20183,RGI60-05.20198,-22.0165,74.9015,5,1,45.809,923,1649,1297,...,20198,05.20198,5.20198,337.9835,5.20198_mb_bins.csv,5.59,2336088,75.00,338.00,7500N-33800E-debris_melt_curve.nc


In [7]:
# Process each group and derive elevation statistics for the debris cover
year_mean = np.zeros((len(lat_values), len(lon_values)))
year_std = np.zeros((len(lat_values), len(lon_values)))
year_med = np.zeros((len(lat_values), len(lon_values)))
year_mad = np.zeros((len(lat_values), len(lon_values)))
doy_mean = np.zeros((len(lat_values), len(lon_values)))
doy_std = np.zeros((len(lat_values), len(lon_values)))
doy_med = np.zeros((len(lat_values), len(lon_values)))
doy_mad = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mean = np.zeros((len(lat_values), len(lon_values)))
dayfrac_std = np.zeros((len(lat_values), len(lon_values)))
dayfrac_med = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mad = np.zeros((len(lat_values), len(lon_values)))


ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)

for nlatlon, ostrem_fn in enumerate(ostrem_fn_all):
# for nlatlon, ostrem_fn in enumerate([ostrem_fn_all[96]]):
    
    main_glac_rgi_subset = main_glac_rgi[main_glac_rgi['ostrem_fn'] == ostrem_fn]
    main_glac_rgi_subset.reset_index(inplace=True, drop=True)
    
    lat_idx = np.where(main_glac_rgi_subset.loc[0,'lat_nearest'] == lat_values)[0][0]
    lon_idx = np.where(main_glac_rgi_subset.loc[0,'lon_nearest'] == lon_values)[0][0]
    
    lat_deg = lat_values[lat_idx]
    lon_deg = lon_values[lon_idx]
    
    print(nlatlon, lat_deg, lon_deg, ostrem_fn)

    doy_list = []
    year_list = []
    dayfrac_list = []
#     print('\n\nDELETE ME: SWITCH BACK!\n\n')
    for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
        glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

#         print(nglac, glac_idx, rgiid)

        # Ts filenames
        ts_fn_idx = np.where(ts_fns_df['RGIId'].values == rgiid)[0][0]
        ts_fn = ts_fns_df.loc[ts_fn_idx,'ts_fullfn']
        ts_year_fn = ts_fns_df.loc[ts_fn_idx,'ts_year_fullfn']
        ts_doy_fn = ts_fns_df.loc[ts_fn_idx,'ts_doy_fullfn']
        ts_dayfrac_fn = ts_fns_df.loc[ts_fn_idx,'ts_dayfrac_fullfn']

        
        # Process debris-covered glaciers
        if (((main_glac_rgi_subset.loc[glac_idx, 'DC_Area_%'] > debris_prms.dc_percarea_threshold) | 
             (main_glac_rgi_subset.loc[glac_idx, 'DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
            and '0.0' not in ts_fns_df.loc[ts_fn_idx].values):

            # Create glacier feature from ice thickness raster
            thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
            thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()
            
            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_dhdt=False, gf_add_vel=False, 
                          gf_add_ts=True, ts_fn=ts_fn,
                          gf_add_ts_info=True, ts_year_fn=ts_year_fn, ts_doy_fn=ts_doy_fn, ts_dayfrac_fn=ts_dayfrac_fn)

            # Isolate the debris-covered areas
            gf.ts = np.ma.array(gf.ts.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts.data)))
            gf.ts_dayfrac = np.ma.array(gf.ts_dayfrac.data, 
                                        mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_dayfrac.data)))
            gf.ts_year = np.ma.array(gf.ts_year.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_year.data)))
            gf.ts_doy = np.ma.array(gf.ts_doy.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_doy.data)))

#             # ===== PLOTS =====
#             # DEM
#             var_full2plot = gf.z1.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
#             # Surface temperature
#             var_full2plot = gf.ts.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
#             # Ts Day fraction
#             var_full2plot = gf.ts_dayfrac.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-dayfrac'], 'inferno', 'ts dayfrac', close_fig=False)
#             # Ts Year
#             var_full2plot = gf.ts_year.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-year'], 'inferno', 'ts year', close_fig=False)
#             # Ts Day of year
#             var_full2plot = gf.ts_doy.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-doy'], 'inferno', 'doy', close_fig=False)
            
#         # ========================================================================================================
                    
#             print(gf.ts_year.compressed())
#             print(gf.ts_doy.compressed())
    
            doy_list.extend(list(gf.ts_doy.compressed()))
            year_list.extend(list(gf.ts_year.compressed()))
            dayfrac_list.extend(list(gf.ts_dayfrac.compressed()))

    
    if len(doy_list) > 0 and len(year_list) > 0 and len(dayfrac_list) > 0:
        # Year
        year_mean_latlon = np.mean(year_list)
        year_std_latlon = np.std(year_list)
        year_med_latlon = malib.fast_median(year_list)
        year_mad_latlon = malib.mad(year_list)
        # Day of year
        #  adjust day of year to avoid issues with comparing JD 1 (Jan 1) and JD 365 (Dec 31),
        #  since the mean would be JD 182 (June), but should be JD 0.5 (Jan 1); also would affect std, etc.
        doy_array = np.array(doy_list)
        doy_array[doy_array > 365] = 365
        doy_array[doy_array > 182] = doy_array[doy_array > 182] - 365
        doy_mean_latlon = np.mean(doy_array)
        if doy_mean_latlon < 0:
            doy_mean_latlon += 365
        doy_std_latlon = np.std(doy_array)
        doy_med_latlon = malib.fast_median(doy_array)
        if doy_med_latlon < 0:
            doy_med_latlon += 365
        doy_mad_latlon = malib.mad(doy_array)
        dayfrac_mean_latlon = np.mean(dayfrac_list)
        dayfrac_std_latlon = np.std(dayfrac_list)
        dayfrac_med_latlon = malib.fast_median(dayfrac_list)
        dayfrac_mad_latlon = malib.mad(dayfrac_list)

        # Update array
        year_mean[lat_idx,lon_idx] = year_mean_latlon
        year_std[lat_idx,lon_idx] = year_std_latlon
        year_med[lat_idx,lon_idx] = year_med_latlon
        year_mad[lat_idx,lon_idx] = year_mad_latlon
        doy_mean[lat_idx,lon_idx] = doy_mean_latlon
        doy_std[lat_idx,lon_idx] = doy_std_latlon
        doy_med[lat_idx,lon_idx] = doy_med_latlon
        doy_mad[lat_idx,lon_idx] = doy_mad_latlon
        dayfrac_mean[lat_idx,lon_idx] = dayfrac_mean_latlon
        dayfrac_std[lat_idx,lon_idx] = dayfrac_std_latlon
        dayfrac_med[lat_idx,lon_idx] = dayfrac_med_latlon
        dayfrac_mad[lat_idx,lon_idx] = dayfrac_mad_latlon

        print('  year mean +/- std:', np.round(year_mean_latlon,1), np.round(year_std_latlon,1)) 
        print('  doy mean +/- std:', np.round(doy_mean_latlon,1), np.round(doy_std_latlon,1)) 
        print('    doy median +/- mad:', np.round(doy_med_latlon,1), np.round(doy_mad_latlon,1)) 
        print('  dayfrac mean +/- std:', np.round(dayfrac_mean_latlon,3), np.round(dayfrac_std_latlon,3))

0 60.0 316.5 6000N-31650E-debris_melt_curve.nc
  year mean +/- std: 2016.0 0.1
  doy mean +/- std: 253.9 0.2
    doy median +/- mad: 253.9 0.0
  dayfrac mean +/- std: 14.133 0.001
1 60.25 316.0 6025N-31600E-debris_melt_curve.nc
  year mean +/- std: 2015.1 1.1
  doy mean +/- std: 239.4 20.0
    doy median +/- mad: 253.9 0.4
  dayfrac mean +/- std: 14.102 0.052
2 60.25 316.75 6025N-31675E-debris_melt_curve.nc
  year mean +/- std: 2014.7 0.9
  doy mean +/- std: 225.8 20.2
    doy median +/- mad: 209.7 4.9
  dayfrac mean +/- std: 14.064 0.049
3 60.5 315.75 6050N-31575E-debris_melt_curve.nc
  year mean +/- std: 2014.7 0.9
  doy mean +/- std: 233.3 67.6
    doy median +/- mad: 210.1 23.4
  dayfrac mean +/- std: 14.115 0.081
4 60.5 316.0 6050N-31600E-debris_melt_curve.nc
  year mean +/- std: 2014.1 0.2
  doy mean +/- std: 209.6 0.4
    doy median +/- mad: 209.6 0.0
  dayfrac mean +/- std: 14.036 0.042
5 60.5 316.25 6050N-31625E-debris_melt_curve.nc
  year mean +/- std: 2014.1 0.4
  doy mean +

  year mean +/- std: 2016.0 0.1
  doy mean +/- std: 202.7 4.1
    doy median +/- mad: 201.0 0.1
  dayfrac mean +/- std: 13.802 0.028
46 66.0 323.25 6600N-32325E-debris_melt_curve.nc
  year mean +/- std: 2016.0 0.0
  doy mean +/- std: 202.8 3.1
    doy median +/- mad: 201.0 0.6
  dayfrac mean +/- std: 13.816 0.037
47 66.0 323.5 6600N-32350E-debris_melt_curve.nc
  year mean +/- std: 2016.0 0.0
  doy mean +/- std: 202.7 2.9
    doy median +/- mad: 201.0 0.6
  dayfrac mean +/- std: 13.816 0.037
48 66.0 323.75 6600N-32375E-debris_melt_curve.nc
  year mean +/- std: 2016.0 0.0
  doy mean +/- std: 202.6 2.6
    doy median +/- mad: 201.0 0.5
  dayfrac mean +/- std: 13.809 0.234
49 66.25 307.25 6625N-30725E-debris_melt_curve.nc
  year mean +/- std: 2015.3 0.7
  doy mean +/- std: 209.0 69.7
    doy median +/- mad: 195.8 1.3
  dayfrac mean +/- std: 15.035 0.083
50 66.25 307.5 6625N-30750E-debris_melt_curve.nc
  year mean +/- std: 2015.1 0.7
  doy mean +/- std: 215.2 85.4
    doy median +/- mad: 19

  year mean +/- std: 2016.0 0.0
  doy mean +/- std: 202.8 6.1
    doy median +/- mad: 203.0 0.0
  dayfrac mean +/- std: 13.563 0.028
91 69.0 332.75 6900N-33275E-debris_melt_curve.nc
  year mean +/- std: 2016.0 0.0
  doy mean +/- std: 356.3 166.4
    doy median +/- mad: 203.8 4.2
  dayfrac mean +/- std: 13.566 0.002
92 69.5 334.75 6950N-33475E-debris_melt_curve.nc
  year mean +/- std: 2015.9 0.3
  doy mean +/- std: 211.8 30.5
    doy median +/- mad: 217.8 8.2
  dayfrac mean +/- std: 13.408 0.073
93 69.75 306.0 6975N-30600E-debris_melt_curve.nc
  year mean +/- std: 2014.7 0.8
  doy mean +/- std: 227.9 101.9
    doy median +/- mad: 189.2 6.8
  dayfrac mean +/- std: 15.198 0.619
94 69.75 306.5 6975N-30650E-debris_melt_curve.nc
  year mean +/- std: 2014.8 0.7
  doy mean +/- std: 223.2 104.7
    doy median +/- mad: 187.2 3.6
  dayfrac mean +/- std: 15.15 1.107
95 69.75 306.75 6975N-30675E-debris_melt_curve.nc
  year mean +/- std: 2015.4 0.6
  doy mean +/- std: 204.5 61.5
    doy median +/- m

  year mean +/- std: 2015.2 0.9
  doy mean +/- std: 216.1 63.1
    doy median +/- mad: 206.0 7.5
  dayfrac mean +/- std: 14.202 0.969
137 74.25 337.75 7425N-33775E-debris_melt_curve.nc
  year mean +/- std: 2015.7 0.5
  doy mean +/- std: 211.3 53.6
    doy median +/- mad: 207.3 3.4
  dayfrac mean +/- std: 13.991 1.186
138 74.5 340.5 7450N-34050E-debris_melt_curve.nc
  year mean +/- std: 2016.0 0.0
  doy mean +/- std: 203.1 1.0
    doy median +/- mad: 203.0 0.0
  dayfrac mean +/- std: 14.029 1.843
139 75.0 338.0 7500N-33800E-debris_melt_curve.nc
  year mean +/- std: 2015.9 0.3
  doy mean +/- std: 222.7 72.2
    doy median +/- mad: 206.2 1.0
  dayfrac mean +/- std: 13.963 0.114
140 75.0 339.75 7500N-33975E-debris_melt_curve.nc
  year mean +/- std: 2015.8 0.5
  doy mean +/- std: 205.2 20.9
    doy median +/- mad: 203.0 2.3
  dayfrac mean +/- std: 13.789 0.324
141 75.25 338.0 7525N-33800E-debris_melt_curve.nc
  year mean +/- std: 2015.9 0.4
  doy mean +/- std: 209.4 31.5
    doy median +/- 

  year mean +/- std: 2015.3 0.4
  doy mean +/- std: 193.1 4.2
    doy median +/- mad: 190.5 1.6
  dayfrac mean +/- std: 18.934 0.425
182 81.75 302.75 8175N-30275E-debris_melt_curve.nc
  year mean +/- std: 2015.0 0.2
  doy mean +/- std: 190.9 8.0
    doy median +/- mad: 189.8 0.0
  dayfrac mean +/- std: 18.757 0.402
183 81.75 303.75 8175N-30375E-debris_melt_curve.nc
  year mean +/- std: 2015.1 0.3
  doy mean +/- std: 193.3 10.5
    doy median +/- mad: 191.8 3.0
  dayfrac mean +/- std: 18.499 0.462
184 81.75 304.0 8175N-30400E-debris_melt_curve.nc
  year mean +/- std: 2015.1 0.3
  doy mean +/- std: 193.1 2.3
    doy median +/- mad: 192.1 2.0
  dayfrac mean +/- std: 18.504 0.316
185 81.75 304.25 8175N-30425E-debris_melt_curve.nc
  year mean +/- std: 2015.4 0.4
  doy mean +/- std: 194.4 3.2
    doy median +/- mad: 194.2 3.6
  dayfrac mean +/- std: 18.887 0.597
186 81.75 304.5 8175N-30450E-debris_melt_curve.nc
  year mean +/- std: 2015.1 0.3
  doy mean +/- std: 192.0 2.2
    doy median +/- 

In [8]:
# Export to dataset
ds_ts_stats = xr.Dataset({'year_mean': (['latitude', 'longitude'], year_mean),
                          'year_std': (['latitude', 'longitude'], year_std),
                          'year_med': (['latitude', 'longitude'], year_med),
                          'year_mad': (['latitude', 'longitude'], year_mad),
                          'doy_mean': (['latitude', 'longitude'], doy_mean),
                          'doy_std': (['latitude', 'longitude'], doy_std),
                          'doy_med': (['latitude', 'longitude'], doy_med),
                          'doy_mad': (['latitude', 'longitude'], doy_mad),
                          'dayfrac_mean': (['latitude', 'longitude'], dayfrac_mean),
                          'dayfrac_std': (['latitude', 'longitude'], dayfrac_std),
                          'dayfrac_med': (['latitude', 'longitude'], dayfrac_med),
                          'dayfrac_mad': (['latitude', 'longitude'], dayfrac_mad),},
                          coords={'latitude': lat_values,
                                  'longitude': lon_values})

attrs_dict={
     'year_mean':{'units':'-',
         'long_name':'mean year',
         'comment': 'mean year when mosaicked surface temperature satellite image was acquired'},
     'year_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of year when mosaicked surface temperature satellite image was acquired'},
     'year_med':{'units':'-',
         'long_name':'median year',
         'comment': 'median year when mosaicked surface temperature satellite image was acquired'},
     'year_mad':{'units':'-',
         'long_name':'median absolute deviation year',
         'comment': 'median absolute deviation of year of when mosaicked surface temperature satellite image was acquired'},
     'doy_mean':{'units':'days since January 1',
         'long_name':'mean day of year',
         'comment': 'mean day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_std':{'units':'days since January 1',
         'long_name':'day of year standard deviation',
         'comment': 'standard deviation of day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_med':{'units':'days since January 1',
         'long_name':'median day of year',
         'comment': 'median day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_mad':{'units':'days since January 1',
         'long_name':'median absolute deviation day of year',
         'comment': 'day of year of year of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mean':{'units':'-',
         'long_name':'mean hour',
         'comment': 'mean hour of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_med':{'units':'-',
         'long_name':'median hour',
         'comment': 'median hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mad':{'units':'-',
         'long_name':'median absolute deviation hour',
         'comment': 'median absolute deviation of hour of when mosaicked surface temperature satellite image was acquired'},}

for vn in ['year_mean', 'year_std', 'year_med', 'year_mad',
           'doy_mean', 'doy_std', 'doy_med', 'doy_mad',
           'dayfrac_mean', 'dayfrac_std', 'dayfrac_med', 'dayfrac_mad',]:
    ds_ts_stats[vn].attrs = attrs_dict[vn]
    
ds_ts_stats.to_netcdf(ts_info_fullfn)
                
print(ds_ts_stats)

<xarray.Dataset>
Dimensions:       (latitude: 92, longitude: 224)
Coordinates:
  * latitude      (latitude) float64 60.0 60.25 60.5 60.75 ... 82.25 82.5 82.75
  * longitude     (longitude) float64 289.2 289.5 289.8 ... 344.5 344.8 345.0
Data variables:
    year_mean     (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_std      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_med      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_mad      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mean      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_std       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_med       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mad       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_mean  (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_std   (latitude

In [9]:
# lat_idx = 4
# lon_idx = 2
# # lat_idx = 37
# # lon_idx = 46
# print(ds_ts_stats['year_mean'][lat_idx,lon_idx].values, ds_ts_stats['year_std'][lat_idx,lon_idx].values, 
#       ds_ts_stats['doy_mean'][lat_idx,lon_idx].values, ds_ts_stats['doy_std'][lat_idx,lon_idx].values)

In [10]:
ts_info_fullfn

'/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/05_ts_data/05_debris_tsinfo.nc'